# Gridworld

In [1]:
import numpy as np
import random

# display output
from random import uniform
import time
from IPython.display import display, clear_output

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) # total number of actions
gridSize = 5 # create a square grid of gridSize by gridSize
state_count = gridSize*gridSize # total number of states

In [3]:
# initialize a policy: create an array of dimension (number of states by number of actions)
# for equal probability amongst all actions, divide everything by the number of actions
policy = np.ones([state_count, action_count]) / action_count

# policy at state 0 = [0, 0]
# returns a probability for each action given state
policy[0]

array([0.25, 0.25, 0.25, 0.25])

In [4]:
class Gridworld():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        self.new_pos = [0, 0] # initialize new position for p_transition
        self.pos_check = [0, 0] # a copy of new position
        self.transition_prob = 1 # deterministic
    
    def initial_state(self):
        # randomly generate an initial state
        i = random.randint(0, len(self.states)-1)
        rand_state = self.states[i]
        return rand_state
    
    def possible_states(self):
        # return the possible states
        return self.states
    
    def reward(self, current_pos, action):
        # return the reward        
        # normally, reward = 0
        reward = 0
        # if taking an action crosses the border = agent stays in same position
#         if -1 in self.pos_check or self.size in self.pos_check: 

        self.new_pos = np.array(current_pos) + np.array(action)
    
        if -1 in self.new_pos or self.size in self.new_pos:
            reward = -1
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            reward = 10
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            reward = 5
        return reward
    
    # def transition_probability(self, current_pos, new_pos):
        # a function that returns the entries of the transition probability matrix?
        # eg. input current state, new state, output = 0.25...0.5...1 ... etc. ?
    
    def p_transition(self, current_pos, action):
        # return the transition probability
        # get next position: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_pos = np.array(current_pos) + np.array(action)
        self.pos_check = self.new_pos # make a copy of new pos before being overwritten below

        # if taking an action crosses the border = agent stays in same position
        if -1 in self.new_pos or self.size in self.new_pos: 
            self.new_pos = current_pos
            
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            self.new_pos = [4, 1]
            
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            self.new_pos = [2, 3]
        return self.new_pos

In [5]:
# create a grid object
grid = Gridworld(5)
grid.valueMap

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [6]:
# # return a random initial state
# grid.initial_state()

In [7]:
# # return all possible states
# grid.possible_states()

## Policy Evaluation

In [8]:
iterations = 0
theta = 0.000001
delta_list = []
discount_factor = 0.99 # small prefer immediate reward, large prefer future reward

In [9]:
# policy evaluation
    # iterate through all 25 states. At each state, iterate through all 4 actions
    # to calculate the value of each action.
    # Replace the value map with the calculated value.
delta_list = []
    
while True:
    delta = 0
    iterations+=1
    valueMap_copy = np.copy(grid.valueMap)

    # start with the first state in the state list
    for state_number, state in enumerate(grid.states):
        value = 0

        # perform 4 actions per state and add the rewards (value)
        for action_number, action in enumerate(actions):

            # get next position and reward
            new_position = grid.p_transition(state, action)
            reward = grid.reward(state, action)

            # calculate value: policy*transition_prob*[r + gamma * value(s')]
            value += policy[state_number][action_number]*grid.transition_prob*(reward+(discount_factor*grid.valueMap[new_position[0], new_position[1]]))          

        # replace the value in valueMap with the value
        valueMap_copy[state[0], state[1]] = value

        # calculate delta
        delta = max(delta, np.abs(value - grid.valueMap[state[0], state[1]]))       
        clear_output(wait=True)
        display('delta: ' + str(delta) + ' iterations: ' + str(iterations))

    # save data for plot
    delta_list.append(delta)

    # overwrite the original value map (update valuemap after one complete iteration of every state)
    grid.valueMap = valueMap_copy

    # stop when change in value function falls below a given threshold
    if delta < theta:
        break

'delta: 9.987400479971598e-07 iterations: 929'

In [10]:
# print value map to 4 decimal places
np.set_printoptions(precision=4)
grid.valueMap

array([[ 3.2175,  7.1243,  4.3031,  4.7526,  1.5245],
       [ 1.4609,  2.7247,  2.2163,  1.7565,  0.3782],
       [-0.4904,  0.2073,  0.1705, -0.2499, -1.1211],
       [-2.1493, -1.5671, -1.4849, -1.8157, -2.5267],
       [-3.467 , -2.9047, -2.7873, -3.0748, -3.7354]])

In [11]:
iterations

929

In [12]:
delta_list

[10.0,
 3.6506250000000002,
 1.7917453125,
 0.7656265546874996,
 0.6238245572753902,
 0.34408380223432605,
 0.2677219248083045,
 0.18628222544686523,
 0.1327107256672725,
 0.10205996706529752,
 0.08747073831669594,
 0.06915746563864489,
 0.057898580618007234,
 0.04695073889257273,
 0.03889143245257154,
 0.032529614278602015,
 0.027331986611153436,
 0.023345019343190998,
 0.01993976322185098,
 0.017553317522488232,
 0.015534580536833431,
 0.013915813181901271,
 0.013257477682216745,
 0.011890551195199883,
 0.011771645683246845,
 0.01085920837245391,
 0.010750616288730086,
 0.010213634239867986,
 0.010011963753179032,
 0.009746328823697858,
 0.009537950733514045,
 0.009331277247349945,
 0.009134117002118636,
 0.008962109582909328,
 0.008788797950425753,
 0.008632428961604433,
 0.008478424693588948,
 0.008336383141515746,
 0.008198193841218782,
 0.008068824965174581,
 0.007943699534628035,
 0.007825321207149294,
 0.0077110549914047866,
 0.007602104324987202,
 0.0074969113282410405,
 0.007

In [18]:
len(delta_list)

929

## Policy Iteration 

In [13]:
# def calculate_action_value(state, value):
#     A = np.zeros(action_count)
    
#     # perform 4 actions per state and add the rewards (value)
#     for action_number, action in enumerate(actions):
            
#         # get next position and reward
#         new_position = grid.p_transition(state, action)
#         reward = grid.reward(state, action)
        
#         # get next position and reward
#         new_position = grid.p_transition(state, action)
#         reward = grid.reward(state, action)

#         # calculate value of action: transition_prob*[r + gamma * value(s')]
#         A[action_number] += grid.transition_prob*(reward+(discount_factor*value[new_position[0], new_position[1]]))
    
#     return A

In [14]:
# # policy improvement

# for state_number, state in enumerate(grid.states):

#     # The best action we would take under the current policy
#     chosen_a = np.argmax(policy[state_number])
# #     print("states: ", state)
# #     print("policy: ", policy[state_number])
# #     print("chosen a: ", chosen_a)

#     # calculate the value of all actions in a given state
#     # eg. action_values = [#, #, #, #] = a value for each of the 4 actions
#     action_values = calculate_action_value(state, grid.valueMap)
# #     print("action values: ", action_values)
    
#     # take the action with the highest value
#     best_a = np.argmax(action_values)

#     # Greedily update the policy
#     if chosen_a != best_a:
#         policy_stable = False

#     # update the policy with the best action
#     print("before policy: ", policy[state_number])
#     policy[state_number] = np.eye(action_count)[best_a]
#     print("after policy: ", policy[state_number])
#     print()

In [15]:
# for state_number, state in enumerate(grid.states):

#     # The best action we would take under the current policy
#     chosen_a = np.argmax(policy[state_number])
#     print("states: ", state)
#     print("policy: ", policy[state_number])
#     print("chosen a: ", chosen_a)

#     action_values = calculate_action_value(state, grid.valueMap)
#     print("action values: ", action_values)
#     best_a = np.argmax(action_values)
    
#     # Greedily update the policy
#     if chosen_a != best_a:
#         policy_stable = False
        
#     print("before policy: ", policy[state_number])
#     policy[state_number] = np.eye(action_count)[best_a]
#     print("after policy: ", policy[state_number])
#     print()

## Testing 

In [16]:
# # get transition: starting in state [0, 0] with action [0, 1], the new state is [0, 1]
# state = [0, 0]
# action = [0, 1] # move right
# grid = Gridworld(5)
# new_position = grid.p_transition(state, action)
# new_position

In [17]:
# # how to solve a system of equations
# a = np.array([[0.5, 0, 0], [-0.5, 1, 0], [0, -0.5, 1]])
# b = np.array([0,0,1])
# x = np.linalg.solve(a,b)
# x